# Logistic regression: Binary classification

In this notebook, we are going to perform binary classification (two classes) using logistic regression

## Dataset - Radar data

This example uses data generated by radars after scanning ionosphere for some time to detect presence of free electrons in ionosphere. Original source of data: https://archive.ics.uci.edu/ml/datasets/Ionosphere

The dataset contains 35 columns - first 34 columns (labelled 0 to 33) are readings of various parameters. The 35th column ( labelled 34) is either 'g' or 'b' indicating 'good' or 'bad', where 'good' means presence of free electrons.

In [157]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as op
import math
from sklearn import preprocessing

%matplotlib inline

In [158]:
data = pd.read_csv('bin_classifier_ionosphere.csv', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [159]:
data.shape

(351, 35)

## How logistic regression works?

In logistic regression (binary classification), we map input feature variables to a discrete output value (either 0 or 1). For doing that, we use [sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function), which is bound between values 0 and 1.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/SigmoidFunction.png/400px-SigmoidFunction.png">

Formally, sigmoid function is:

$$
h(z) = \frac {1} {1-e^{-z}}
$$

So, we map input features to output features by first calculating

$$
z = \theta^T X
$$

and then applying sigmoid function to $z$

The sigmoid is a continuous function, so we obtain discrete output from it as follows:

$$
\begin{align}
h(z) \ge 0.5 \rightarrow y = 1 \\
h(z) \lt 0.5 \rightarrow y = 0
\end{align}
$$

The following function implements a hypothesis for logistic regression:

In [160]:
# Sigmoid function
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Hypothesis function
def hypothesisLogisticRegression(theta, X):
    sig = np.vectorize(sigmoid)
    G = np.dot(X, theta)
    theta = theta.reshape(X.shape[1], 1) # optimizer rehsapes it to row vector, so we reshape it back to column vector
    return sig(np.dot(X, theta))

## Evaluating the hypothesis - The cost function

Any value of $\theta$ represents some decision boundary. The cost function should evaluate how good a given value of $\theta$ is by using that on training examples and calculating how far the hypothesis is from the ground truth values. For a given training example, when we apply our hypothesis function, we are going to get continous value between 0 and 1. If the ground truth is 0 for that example, we should penalize more if the hypothesis is close to 1, and if the ground truth is 1, we should penalize the hypothesis if it is closer to 0. Moreover the penalty should grow exponentially with distance from ground truth.

Therefore, in order to get an exponentially growing penalty, we use the $log$ function, which has the following property:

$-log(x)$ is an exponentially decreasing function, which intercepts x axis at x = 1.

<img src = "http://www.rapidtables.com/math/algebra/logarithm/log-graph.png" />

So if we say $cost = -log(x)$, then for ground truth value 1, we get cost = 0 for x = 1 and cost = $+\infty$ for x = 0.

Similarly, we cau use the same function for ground truth value 0 by saying $cost = -log(1-x)$.

Therefore our cost function looks like:

$$
J = 
\begin{cases}
-log(h_\theta(x)),  & \text{if y = 1} \\
-log(1 - h_\theta(x)), & \text{if y = 0}
\end{cases}
$$

We can even combine the two conditions and have a simplified version of the cost function as follows:

$$
\begin{align}
J & = - ylog(h_\theta(x)) - (1-y)log(1 - h_\theta(x)) \\
& = - [ylog(h_\theta(x)) + (1-y)log(1 - h_\theta(x))]
\end{align}
$$

For all training examples, the cost function would become:

$$
J = -\frac1m \sum_{i=1}^m {y^{(i)}.log(h_\theta(x^{(i)}) + (1-y^{(i)})log(1 - h_\theta(x^{(i)}) }
$$

The following code implements this function:

In [161]:
# Logistic regression cost function
def costLogisticRegression(theta, X, y):
    H = hypothesisLogisticRegression(theta, X)
    m, n = X.shape
    theta = theta.reshape(n, 1) # optimizer rehsapes it to row vector, so we reshape it back to column vector
    J = (-1/m) * ( np.dot(y.T, np.log(H)) + np.dot((1-y).T, np.log(1 - H)) )
    return J

## Finding optimal values of coefficients $\theta$

Just like [Linear regression]("../linear_regression/1_multiple_variables.ipynb"), we can apply gradient descent to logistic regression also.

Gradient descent in this case is given by:

repeat till convergence: {
$$
\begin{align}
\theta_0 := \theta_0 - \alpha \frac 1m \sum_{1=1}^m ( h_\theta(x^{(i)}) - y^{(i)}).x_0^{(i)} \\
\theta_1 := \theta_0 - \alpha \frac 1m \sum_{1=1}^m ( h_\theta(x^{(i)}) - y^{(i)}).x_1^{(i)} \\
\theta_2 := \theta_0 - \alpha \frac 1m \sum_{1=1}^m ( h_\theta(x^{(i)}) - y^{(i)}).x_2^{(i)} \\
... \\
\theta_n := \theta_0 - \alpha \frac 1m \sum_{1=1}^m ( h_\theta(x^{(i)}) - y^{(i)}).x_n^{(i)} \\
\end{align}
$$
}

Instead of manually iterating and reaching to min value of cost function, we can also use built in functions provided by scipy library to get the optimal values of $\theta$. We are going to do that in this example:

In [162]:
# Gradient function to be used by optimizer
def gradLogisticRegression(theta, X, y):
    H = hypothesisLogisticRegression(theta, X)
    m, n = X.shape
    grad = (1/m) * (np.dot( (H - y).T , X ).T)
    return grad

# Gradient descent - optimizes cost
def gradientDescentLogisticRegression(theta_init, X, y, maxiter):
    res = op.minimize(fun = costLogisticRegression, 
                                 x0 = theta_init, 
                                 args = (X, y),
                                 method = 'TNC',
                                 jac = gradLogisticRegression,
                                 options = {
                                    'maxiter': maxiter,
                                    'disp': True #  No idea why its not working
                                    }
                                 );
    return res.x # optimal theta

## Applying algorithm on data

### Step 1: Extract data into matrices

In [163]:
# Extract into separate matrices for input and output
data = np.matrix(data)

# Important to shuffle data before beginning, as input data may be biased towards
# some particular classes in the beginning - depending on the method used to obtain examples
np.random.shuffle(data) 

X = data[:,0:34]
y = data[:,34]
y = np.reshape(y, (y.shape[0], 1))

y = (y == 'g') # 'g' -> 1, other values -> 0

### Step 2: Partitioning and scaling

We will partition the sample data into two parts - training set and test set. And then use scikit's scaling function to bring all data to same mean and comparable variance.

In [164]:
total_size = X.shape[0]
training_set_size = int(total_size * (0.8))
test_set_size = total_size - training_set_size

Xtrain = X[0:training_set_size, :]
XtrainNorm = preprocessing.scale(Xtrain.astype(np.float64)) # scale to mean = 0 and unit variance
XtrainNorm = np.append(np.ones((XtrainNorm.shape[0],1)), XtrainNorm, axis = 1)
ytrain = y[0:training_set_size]

Xtest = X[(training_set_size):(training_set_size + test_set_size), :]
XtestNorm = preprocessing.scale(Xtest.astype(np.float64)) # scale to mean = 0 and unit variance
XtestNorm = np.append(np.ones((XtestNorm.shape[0],1)), XtestNorm, axis = 1)
ytest = y[(training_set_size): (training_set_size + test_set_size)]

### Step 3: Applying gradient descent to minimize cost

We'll call the functions defined above for our data.

In [185]:
theta_init = np.zeros((XtrainNorm.shape[1], 1)) # start from zeros

maxiter = 100 # Obtained from trial and error
theta = gradientDescentLogisticRegression(theta_init, XtrainNorm, ytrain, maxiter)

J_training = costLogisticRegression(theta, XtrainNorm, ytrain)
J_test = costLogisticRegression(theta, XtestNorm, ytest)

print("Training error (" + str(training_set_size) + " examples)", J_training)
print("Test error (" + str(test_set_size) + " examples)", J_test)

Training error (280 examples) [[ 0.15395152]]
Test error (71 examples) [[ 0.56039339]]


###  Step 4: Calculating accuracy

In case of classficiation problems, the end goal is to be able to predict class = 0 or class = 1 for any given input. In our case, when we apply hypothesis function, we are getting a continuous value. So we need to convert the hypothesis into prediction by taking any value greater than a _threshold_ to be a positive hypothesis.

Following code performs that and calculated accuracy in percentage:

In [196]:
def calculateAccuracy(theta, X, y, threshold):
    m, n = X.shape
    H = hypothesisLogisticRegression(theta, X)
    
    # Predicting anything greater than 'threshold' as positive
    P = np.zeros((m, 1))
    P[H > threshold] = 1
    P[H <= threshold] = 0
    
    match = np.zeros((m, 1))
    match[P == y] = 1
    
    return (sum(match) * 100 / m) # percentage accuracy

And now we can calculate accuracy of our model for training and test sets.

In [199]:
# For simple logistic regression, threshold = 0.5 (see How logistic regression works?" above)
a_training = calculateAccuracy(theta, XtrainNorm, ytrain, 0.5)
a_test = calculateAccuracy(theta, XtestNorm, ytest, 0.5)

print("Training accuracy (percentage)", a_training)
print("Test accuracy (percentage)", a_test)

Training accuracy (percentage) [ 93.57142857]
Test accuracy (percentage) [ 91.54929577]
